In [35]:
%matplotlib inline
import pandas as pd
import matplotlib
import time
import numpy as np
import matplotlib.pyplot as plt
TRADE_DATA = "../data/Trade.csv"

In [36]:
df_trade = pd.read_csv(TRADE_DATA)
df_trade.columns

Index(['TradeDateKey', 'CustomerIdx', 'IsinIdx', 'BuySell', 'NotionalEUR',
       'Price', 'TradeStatus', 'CustomerInterest'],
      dtype='object')

In [37]:
df_trade = df_trade[df_trade["TradeDateKey"] >= 20180101]
# Delete Holding Values
df_trade = df_trade[df_trade["TradeStatus"] != "Holding"]

# Drop Useless Columns
df_trade = df_trade.drop(["TradeStatus", "NotionalEUR", "Price"], axis=1)
df_trade = df_trade.sort_values("TradeDateKey", ascending=True)

# Take 1 week
df_trade = df_trade[(df_trade["TradeDateKey"] >= 20180328) & (df_trade["TradeDateKey"] < 20180405)]

In [38]:
df_trade = df_trade[(df_trade["TradeDateKey"] >= 20180328) & (df_trade["TradeDateKey"] < 20180405)]
df_trade = df_trade.groupby(["CustomerIdx", "IsinIdx"]).count()
df_trade["CustomerIdx"] = df_trade["CustomerInterest"].apply(lambda x: 1 if x > 1 else x)

In [40]:
df_trade.head()

TradeDateKey  BuySell  CustomerInterest  CustomerIdx
CustomerIdx IsinIdx                                                      
2           5215                2        2                 2            1
            16812               2        2                 2            1
            17091               1        1                 1            1
            17275               2        2                 2            1
14          22209               2        2                 2            1

In [11]:
df_trade.groupby("TradeDateKey").count()["CustomerInterest"].values

array([     1,   3524,   5317,   4808,   5031,      2,     19,   5549,
         4622,   5779,   5827,   5731,      5,   1951,   4172,   5096,
         6523,   4711,      7,   4919,   5332,   4708,   6833,   4561,
           15,   5881,   7591, 197510,   6383,   5156,      5,   6088,
         6513,   6159,   7461,   6590,      2,   6710,   6266,   5346,
         5850,   4414,     10,   1618,   5162,   5078,   7130,   4732,
            2,      6,   7591,   5091, 120770,   5307,   4602,   4970,
         5240,   4535,   4108,   3815,   5219,   4615,   4993,   5917,
         4180,   6051,  10461,   6197,   5355,   1878,      1,      1,
         6989,   5462,   6153,   9472,     97, 117437,   1793,   5577,
         4742,  10166,   5414,      1,     14,   3949,   4809,   6407,
         5500,   5401,     49,   4722,   5455,   5987,   5140,   4768,
            3])

In [8]:
dictionary = df_trade.groupby(['CustomerIdx', 'IsinIdx']).count()['TradeDateKey'].to_dict()
dictionary[(0, 24944)]

1

In [41]:
df_trade.describe()
print(np.sort(df_trade["TradeDateKey"].unique()))

TEST_DATE = 20180412
VAL_DATE = 20180405
TRAIN_DATE = 20180328

[20180101 20180102 20180103 20180104 20180105 20180106 20180107 20180108
 20180109 20180110 20180111 20180112 20180114 20180115 20180116 20180117
 20180118 20180119 20180121 20180122 20180123 20180124 20180125 20180126
 20180128 20180129 20180130 20180131 20180201 20180202 20180204 20180205
 20180206 20180207 20180208 20180209 20180211 20180212 20180213 20180214
 20180215 20180216 20180218 20180219 20180220 20180221 20180222 20180223
 20180224 20180225 20180226 20180227 20180228 20180301 20180302 20180305
 20180306 20180307 20180308 20180309 20180312 20180313 20180314 20180315
 20180316 20180319 20180320 20180321 20180322 20180323 20180324 20180325
 20180326 20180327 20180328 20180329 20180330 20180331 20180402 20180403
 20180404 20180405 20180406 20180407 20180408 20180409 20180410 20180411
 20180412 20180413 20180415 20180416 20180417 20180418 20180419 20180420
 20180422]


In [59]:
# Create a Double KEY dictionary
"""train = df_trade.values

from collections import defaultdict
from itertools import product

customer_ids = np.unique(train[:,1])
bonds_ids = np.unique(train[:,2])

cbidx_freq = defaultdict(dict)
for c, b in product(customer_ids, bonds_ids):
    for sample in train:
        i = 0
        if sample[1] == c and sample[2] == b:
            i = i + 1
        cbidx_freq[c][b] = i"""
"""max_value = df_trade["TradeDateKey"].max()
df_trade[df_trade["TradeDateKey"] == max_value].groupby("CustomerIdx").count()["CustomerInterest"]"""


# Generate the label
df_trade.groupby(['CustomerIdx','IsinIdx']).count()['TradeDateKey']

# Generate the train set
df_trade[(df_trade["TradeDateKey"] >= TRAIN_DATE) & (df_trade["TradeDateKey"] < VAL_DATE)].groupby(["CustomerIdx", "IsinIdx"]).count().describe()
df_trade[(df_trade["TradeDateKey"] >= VAL_DATE) & (df_trade["TradeDateKey"] < TEST_DATE)].groupby(["CustomerIdx", "IsinIdx"]).count().describe()
df = df_trade[df_trade["TradeDateKey"] >= TEST_DATE].groupby(["CustomerIdx", "IsinIdx"]).count()["CustomerInterest"]

print(df.head())
print(df.values[0:5])
df.reset_index(level=['CustomerIdx', 'IsinIdx'])

CustomerIdx  IsinIdx
0            27045      1
             27298      1
5            10842      1
8            1812       1
             3364       1
Name: CustomerInterest, dtype: int64
[1 1 1 1 1]


,CustomerIdx,IsinIdx,CustomerInterest
0,0,27045,1
1,0,27298,1
2,5,10842,1
3,8,1812,1
4,8,3364,1
5,8,21277,1
6,8,24116,1
7,8,25460,1
8,8,26341,1
9,9,7504,1


In [6]:
from itertools import islice
dictionary = df_trade.groupby('IsinIdx').count()['TradeDateKey'].to_dict()

def take(n, iterable):
    return list(islice(iterable, n))
def print_dictionary(dictionary=None):
    n_items = take(10, dictionary.items())
    for key, val in n_items:
        print(key, val)
    return
print_dictionary(dictionary)

1 7
2 10
3 1
4 4
5 1
6 40
7 76
14 2
15 8
17 31
